In [4]:
DATA_LOCATION = "/work/mk423/Fear Conditioning/"
UMC_PATH = "/hpc/home/mk423/Anxiety/Universal-Mouse-Code/"
MODEL_PATH = "../Models/"
data_file = "FearConditioning_Data.pkl"

FEATURE_LIST = ['X_psd','X_coh','X_gc']
FEATURE_VECTOR = FEATURE_LIST
FEATURE_WEIGHT = [10,1,1]

MT_MODEL_PATH = MODEL_PATH + "Positive_MT_10_res_loss_10_power_features.pt"
PROJECTION_SAVE_PATH = "/hpc/home/mk423/Anxiety/MultiTaskWork/Projections/holdoutExperiments/"

TRAIN = False
PROJ_TEST = True

In [2]:
import pickle
import numpy as np
import torch
import matplotlib.pyplot as plt
import os
import sys
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
sys.path.append(UMC_PATH)
sys.path.append(PROJECTION_SAVE_PATH)
#from dCSFA_model import dCSFA_model
import umc_data_tools as umc_dt
from dCSFA_NMF import dCSFA_NMF

if torch.cuda.is_available():
    device="cuda:0"
else:
    device="cpu"


print("Using device: %s"%(device))

#For Consistency
RANDOM_STATE=42

import pandas as pd

model = torch.load(MT_MODEL_PATH,map_location='cpu')
model.device = "cpu"
model.eval()

/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda:0


dCSFA_NMF(
  (recon_loss_f): MSELoss()
  (Encoder): Sequential(
    (0): Linear(in_features=5152, out_features=256, bias=True)
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): Linear(in_features=256, out_features=20, bias=True)
    (4): Softplus(beta=1, threshold=20)
  )
)

In [10]:
dataDict = pickle.load(open(DATA_LOCATION + data_file,"rb"))

X = np.hstack([weight*dataDict[feature] for weight,feature in zip(FEATURE_WEIGHT,FEATURE_LIST)])
y_time = dataDict['y_time']
y_mouse = dataDict['y_mouse'].squeeze()
y_expDate = dataDict['y_expDate']
y_fwt = dataDict['y_freeze_w_tone']
y_fo = dataDict['y_freeze_only']
y_to = dataDict['y_tone_only']

In [19]:
y_pred,s = model.transform(X,None)[2:]

#Get per mouse average scores, average scores in HC and average scores in EPM
mouse_list = []
avg_score_list = []
avg_hc_score_list = []
avg_task_score_list = []
condition_list = []
for mouse in np.unique(y_mouse):
    mouse_mask = y_mouse==mouse
    hc_mouse_mask = np.logical_and(mouse_mask,y_fwt.squeeze()==0)
    task_mouse_mask = np.logical_and(mouse_mask,y_fwt.squeeze()==1)
    #print("Mouse: {} non cond size: {}, cond size: {}".format(mouse,np.unique(hc_mouse_mask),np.unique(task_mouse_mask)))
    avg_score = np.mean(s[mouse_mask==1,0])
    avg_score_hc = np.mean(s[hc_mouse_mask==1,0])
    avg_score_task = np.mean(s[task_mouse_mask==1,0])
    
    mouse_list.append(mouse)
    avg_score_list.append(avg_score)
    avg_hc_score_list.append(avg_score_hc)
    avg_task_score_list.append(avg_score_task)
    
proj_dict = {
    "mouse":mouse_list,
    "avgScore":avg_score_list,
    "avg_non_condition_Score":avg_hc_score_list,
    "avg_freezing_w_tone_Score":avg_task_score_list
}
df_projections = pd.DataFrame.from_dict(proj_dict)

df_projections.to_csv(PROJECTION_SAVE_PATH + "FC_mean_scores.csv")

In [24]:
mw_auc_dict = umc_dt.lpne_auc(y_pred,y_fwt.squeeze(),y_mouse,s,True)
mw_mean, mw_std = umc_dt.get_mean_std_err_auc(y_pred,y_fwt.squeeze(),y_mouse,s,True)
print("by freezing with tone auc: {:.3} +/- {:.3}".format(mw_mean,mw_std))

mw_auc_dict = umc_dt.lpne_auc(y_pred,y_fo.squeeze(),y_mouse,s,True)
mw_mean, mw_std = umc_dt.get_mean_std_err_auc(y_pred,y_fo.squeeze(),y_mouse,s,True)
print("by freezing only auc: {:.3} +/- {:.3}".format(mw_mean,mw_std))

mw_auc_dict = umc_dt.lpne_auc(y_pred,y_to.squeeze(),y_mouse,s,True)
mw_mean, mw_std = umc_dt.get_mean_std_err_auc(y_pred,y_to.squeeze(),y_mouse,s,True)
print("by tone only auc: {:.3} +/- {:.3}".format(mw_mean,mw_std))

Mouse  Mouse04191  has only one class - AUC cannot be calculated
n_positive samples  0
n_negative samples  1020
Mouse  Mouse9001  has only one class - AUC cannot be calculated
n_positive samples  0
n_negative samples  1004
Mouse  Mouse9003  has only one class - AUC cannot be calculated
n_positive samples  0
n_negative samples  779
Mouse  Mouse9013  has only one class - AUC cannot be calculated
n_positive samples  0
n_negative samples  1015
Mouse  Mouse9021  has only one class - AUC cannot be calculated
n_positive samples  0
n_negative samples  1002
Mouse  Mouse9022  has only one class - AUC cannot be calculated
n_positive samples  0
n_negative samples  1000
Mouse  Mouse9064  has only one class - AUC cannot be calculated
n_positive samples  0
n_negative samples  995
Mouse  Mouse04191  has only one class - AUC cannot be calculated
n_positive samples  0
n_negative samples  1020
Mouse  Mouse9001  has only one class - AUC cannot be calculated
n_positive samples  0
n_negative samples  1004
M

In [26]:
mw_auc_dict = umc_dt.lpne_auc(y_pred,y_fwt.squeeze(),y_mouse,s,True)
mw_mean, mw_std = umc_dt.get_mean_std_err_auc(y_pred,y_fwt.squeeze(),y_mouse,s,True)
print("by freezing with tone auc: {:.3} +/- {:.3}".format(mw_mean,mw_std))

auc_list = []
p_val_list = []

for mouse in y_mouse:
    auc_list.append(mw_auc_dict[mouse][0])
    p_val_list.append(mw_auc_dict[mouse][1])

saveDict = {
    "mouse":y_mouse.squeeze(),
    "freeze with tone":y_fwt.squeeze(),
    "freeze only":y_fo.squeeze(),
    "tone only":y_to.squeeze(),
    "time":y_time.squeeze(),
    "expDate":y_expDate.squeeze(),
    "scores":s[:,0],
    "freeze with tone auc": auc_list,
    "freeze with tone pval": p_val_list,
    
}

df = pd.DataFrame.from_dict(saveDict)
df.to_csv(PROJECTION_SAVE_PATH + "MT_onto_FC.csv")

Mouse  Mouse04191  has only one class - AUC cannot be calculated
n_positive samples  0
n_negative samples  1020
Mouse  Mouse9001  has only one class - AUC cannot be calculated
n_positive samples  0
n_negative samples  1004
Mouse  Mouse9003  has only one class - AUC cannot be calculated
n_positive samples  0
n_negative samples  779
Mouse  Mouse9013  has only one class - AUC cannot be calculated
n_positive samples  0
n_negative samples  1015
Mouse  Mouse9021  has only one class - AUC cannot be calculated
n_positive samples  0
n_negative samples  1002
Mouse  Mouse9022  has only one class - AUC cannot be calculated
n_positive samples  0
n_negative samples  1000
Mouse  Mouse9064  has only one class - AUC cannot be calculated
n_positive samples  0
n_negative samples  995
Mouse  Mouse04191  has only one class - AUC cannot be calculated
n_positive samples  0
n_negative samples  1020
Mouse  Mouse9001  has only one class - AUC cannot be calculated
n_positive samples  0
n_negative samples  1004
M